# DPKE_NTRU_HPS2048509

Esse trabalho foi desenvolvido com base no documento "NTRU Algorithm Specifications And Supporting Documentation".

In [122]:
import random
import unittest

 criar a base de polinômios; $\mathbb{Z}$ é o polinômio de coeficientes inteiros, de forma que $\mathbb{Z}/2\mathbb{Z}$, $\mathbb{Z}/3\mathbb{Z}$, $\mathbb{Z}/q\mathbb{Z}$, são inteiros módulo 2, 3 e q respectivamente.

In [123]:
q=2048
Z.<xZ> = ZZ[]              # ZZ[x]
Z2.<xZ2> = Integers(2)[]   # (ZZ/2ZZ)[x]
Z3.<xZ3> = Integers(3)[]   # (ZZ/3ZZ)[x]
Zq.<xZq> = Integers(q)[]   # (ZZ/2048ZZ)[x]
# xZ = Z.gen()
# xZ.content()
print( xZ.parent())
print(xZ2.parent())
print(xZ3.parent())
print(xZq.parent())

Univariate Polynomial Ring in xZ over Integer Ring
Univariate Polynomial Ring in xZ2 over Ring of integers modulo 2 (using GF2X)
Univariate Polynomial Ring in xZ3 over Ring of integers modulo 3
Univariate Polynomial Ring in xZq over Ring of integers modulo 2048


confirmada a base do anel, implementamos a convolução cíclica, utilizando funções nativas do SageMath

In [124]:
n= 509
Phi_n = lambda x : (x^n-1)//(x-1)
Phi_1 = lambda x : (x - 1)
R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
# R1.<xR1> = Zq.quotient(Phi_1(xZq))

print("R:", xR.parent())
print("S:", xS.parent())
print("R3:", xR3.parent())
print("Rq:", xRq.parent()) 
print("S2:", xS2.parent())
print("S3:", xS3.parent())
print("Sq:", xSq.parent())

R: Univariate Quotient Polynomial Ring in xR over Integer Ring with modulus xZ^509 - 1
S: Univariate Quotient Polynomial Ring in xS over Integer Ring with modulus xZ^508 + xZ^507 + xZ^506 + xZ^505 + xZ^504 + xZ^503 + xZ^502 + xZ^501 + xZ^500 + xZ^499 + xZ^498 + xZ^497 + xZ^496 + xZ^495 + xZ^494 + xZ^493 + xZ^492 + xZ^491 + xZ^490 + xZ^489 + xZ^488 + xZ^487 + xZ^486 + xZ^485 + xZ^484 + xZ^483 + xZ^482 + xZ^481 + xZ^480 + xZ^479 + xZ^478 + xZ^477 + xZ^476 + xZ^475 + xZ^474 + xZ^473 + xZ^472 + xZ^471 + xZ^470 + xZ^469 + xZ^468 + xZ^467 + xZ^466 + xZ^465 + xZ^464 + xZ^463 + xZ^462 + xZ^461 + xZ^460 + xZ^459 + xZ^458 + xZ^457 + xZ^456 + xZ^455 + xZ^454 + xZ^453 + xZ^452 + xZ^451 + xZ^450 + xZ^449 + xZ^448 + xZ^447 + xZ^446 + xZ^445 + xZ^444 + xZ^443 + xZ^442 + xZ^441 + xZ^440 + xZ^439 + xZ^438 + xZ^437 + xZ^436 + xZ^435 + xZ^434 + xZ^433 + xZ^432 + xZ^431 + xZ^430 + xZ^429 + xZ^428 + xZ^427 + xZ^426 + xZ^425 + xZ^424 + xZ^423 + xZ^422 + xZ^421 + xZ^420 + xZ^419 + xZ^418 + xZ^417 + xZ^416 + 

O resultado comprova que tanto o grau quanto as reduções de cada anel estão corretos. Entretanto de acordo com o artigo $Rq$, $Sq$ e $S3$ possuem coeficientes centralizados entre $- q/2$ até $q/2-1$,  assim o resultado até aqui garante que ainda não foi obtida a representação correta.

#### Representantes canônicos

O Sage por padrão gera representantes canônicos com coeficientes dentro de intervalo não negativo $[0, q-1]$, não há uma função nativa que possa mudar esse comportamento. Os representantes canônicos definidos no artigo são específicos para o NTRU. Dessa forma foi implementada a função canonicalNTRU para converter coeficientes para o intervalo centrado.
$$
\begin{align*}
    \underline{R_q} \quad & \text{com grau } n-1 \text{ coeficientes dentro do intervalo } \left[ -q/2, q/2-1\right]. \\
    \underline{S_q} \quad & \text{com grau } n-2 \text{ coeficientes dentro do intervalo } \left[-q/2, q/2-1\right]. \\
    \underline{S3} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}.
\end{align*}
$$

In [125]:
def canonicalNTRU(ring, coeffs, n, q):
    a = ring(coeffs)

    if ring == S3:
        coeffs_can = twos_complement_S3(a, n)

    elif ring == Rq:
        coeffs_can = twos_complement_coeffs(a, n, q)
    
    else:
        coeffs_can = twos_complement_coeffs(a, n, q)
    Z.<xZ> = ZZ[] # novo anel que o elemento pertencerá

    return Z(coeffs_can)

def twos_complement_S3(a, n):
    coeffs = [0] * n

    for i, ai in enumerate(a):
        coeffs[i % n] += int(ai)

    def rep_S3(c):
        r = c % 3
        if r == 2:
            return -1
        return r

    return [rep_S3(c) for c in coeffs]

def twos_complement_coeffs(a, n, q):
    coeffs = [0]*n
    for i, ai in enumerate(a):
        if i < n:
            coeffs[i] += int(ai)

    def rep_centered(val):
        r = int(val) % q
        if r > q//2 or (q % 2 == 0 and r == q//2):
            r = r - q
        return r

    return [rep_centered(c) for c in coeffs]
    

Apesar de assumirmos que estamos lidando com o representante canônico, o SageMath entende que o polinômio retornado pela função canonical não pertence mais ao anel do polinômio de entrada da função. Para que fosse possível manter a coerção seria necessário inicializar todas os polinômios em base $\mathbb{Z}[\mathbf{x}]$ sem redução, realizando as reduções "manualmente", entretanto também seria necessário uma função para substituir e manter a eficiencia.

Dessa forma para evitar erros de coersão entre anéis diferentes, utilizaremos uma função que passa o polinômio para base $\mathbb{Z}$, isso equivale a Z(a.list())

In [126]:
def zz(a):
    Z.<xZ> = ZZ[]
    poly = Z(a.list())
    return poly

#### Sample $\mathcal{T}$ and $\mathcal{T(d)}$

Dado $g$ amostrado de $\mathcal{L}_g=\mathcal{T}$($d$), dessa forma definimos $d=\frac{q}{8}-2$
Onde $0 < d \leq \frac{2n}{3}$. Nesse contexto $d/2$ será o valor que define a quantidade de 1 e -1, que devem ser a mesma quantidade, o resto dos coeficientes será zero. Dessa forma definimos as condições para $q$ que deve ser potência de 2, consequentemente, inteiro e par. garante que $gcd(p,q)=1$. Após definir $n$ podemos definir $q$ a partir da desigualdade;
\begin{equation}
    \frac{q}{8}-2 \leq \frac{2n}{3} \quad\longrightarrow \quad q \leq 8 \bigg(\frac{2n}{3}+2\bigg)
\end{equation}
pode escolher até a maior potência de 2 menor ou igual a esse limite. Caso ainda haja interesse em usar um $q$ maior, a especificação recomenda fixar $d$ em $2 \lfloor n/3\rfloor$. $q$ aumenta o tamanho da chave, impactando o trade-off entre segurança e eficiência.

Definimos os polinômios ternários $\mathcal{T}$ e $\mathcal{T}(d)$, que serão usados tanto na geração das chaves $f$ e $g$, quanto para $r$ e $m$;
\begin{align*}
    \mathcal{T} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    \mathcal{T}(d) \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

De acordo com as específicações do hps2048509 adotamos $p=3$, e $n=509$, consequentemente $q=2048$ sendo a maior potência de $2$ abaixo do limite em (1). Portanto $\mathcal{T}(d)$ deve ter 127 coeficientes 1's e 127 coeficientes -1's, restando 254 zeros.

In [127]:
def ternary(n):
    return  Z(random.choices([-1, 0, 1], k=n-1))

def ternary_fixed(n,q):
    d = q//8-2
    ind = list(range(n-1))
    random.shuffle(ind)
    pos = ind[0:d//2]
    neg = ind[d//2:d]
    c = [0]*(n-1)

    for i in pos:
        c[i] = 1
    for i in neg:
        c[i] = -1
    return Z(c)

In [128]:
def count(poly):
    coeffs = poly.list()
    zeros = coeffs.count(0)
    ones = coeffs.count(1)
    neg_ones = coeffs.count(-1)
    return zeros, ones, neg_ones

In [129]:
g = ternary_fixed(n, q)
print("g: ", g)
print("g_in_S3", canonicalNTRU(S3, g.list(), n, q))

zeros, ones, neg_ones = count(g)

print(f"Zeros: {zeros}")
print(f"Uns: {ones}")
print(f"Menos Uns: {neg_ones}")

g:  -xZ^507 - xZ^506 + xZ^505 + xZ^504 + xZ^501 + xZ^500 + xZ^499 - xZ^498 - xZ^492 - xZ^491 + xZ^489 - xZ^488 - xZ^487 - xZ^486 - xZ^485 + xZ^484 - xZ^477 + xZ^472 + xZ^470 - xZ^469 + xZ^467 - xZ^464 - xZ^462 - xZ^458 - xZ^457 - xZ^456 - xZ^453 + xZ^452 + xZ^451 + xZ^449 - xZ^445 - xZ^444 + xZ^443 + xZ^441 + xZ^439 + xZ^438 + xZ^437 + xZ^436 - xZ^435 + xZ^434 - xZ^433 - xZ^431 + xZ^429 + xZ^426 - xZ^425 + xZ^424 - xZ^423 + xZ^421 - xZ^420 + xZ^413 + xZ^409 - xZ^407 - xZ^404 - xZ^402 - xZ^401 + xZ^398 - xZ^396 - xZ^394 - xZ^390 - xZ^389 - xZ^387 - xZ^385 + xZ^379 + xZ^377 + xZ^375 - xZ^373 + xZ^369 - xZ^368 - xZ^367 + xZ^366 + xZ^365 - xZ^362 + xZ^361 - xZ^360 - xZ^359 - xZ^358 + xZ^356 + xZ^355 - xZ^354 - xZ^353 - xZ^349 + xZ^348 - xZ^345 - xZ^344 + xZ^343 - xZ^340 + xZ^339 - xZ^338 - xZ^337 - xZ^335 + xZ^328 - xZ^327 - xZ^325 - xZ^321 + xZ^320 - xZ^318 + xZ^317 - xZ^312 + xZ^311 - xZ^310 + xZ^308 - xZ^307 - xZ^303 + xZ^302 + xZ^301 - xZ^300 + xZ^297 - xZ^296 + xZ^294 + xZ^290 - xZ^28

O core do método parte da representação de qualquer polinômio de grau 512 como; 
\begin{equation}
    a(x) = \sum_{i=0}^{511}a_i \cdot x^i = \underbrace{\bigg(\sum_{i=0}^{255}a_i \cdot x^i\bigg)}_{a_L} + x^{256}\cdot \underbrace{\bigg(\sum_{i=0}^{255}a_{i+256} \cdot x^i\bigg)}_{a_H}
\end{equation}

In [130]:
def mul_primitive(a, b):
    return a*b

def poly_512(a):
    return a + [0]*3

def divide_poly(a):
    n = len(a)
    m = n // 2    

    aL = a[0:m] # metade low  de 0 a 255
    aH = a[m:n] # metade high de 0 a 255

    return aL, aH

# def add_poly(a, b):
#     return [a[i] + b[i] for i in range(len(a))]
def add_poly(a, b):
    n = max(len(a), len(b))
    res = [0]*n
    for i in range(n):
        if i < len(a):
            res[i] += a[i]
        if i < len(b):
            res[i] += b[i]
    return res


def shift_poly(a, k):
    return [0]*k + a

def mul(a, b):
    n = len(a)
    
    if n <= 64:
        return mul_primitive(a, b)
    else:
        # divide
        aL, aH = divide_poly(a)
        bL, bH = divide_poly(b)

        m = n//2

        # produtos recursivos
        p0 = mul(aL, bL)   # aL bL
        p1 = mul(aL, bH)   # aL bH
        p2 = mul(aH, bL)   # aH bL
        p3 = mul(aH, bH)   # aH bH

        dist = add_poly(p1, p2)

        result = add_poly(
            add_poly(p0, shift_poly(dist, m)),
            shift_poly(p3, 2*m)
        )
        return result

### Cálculo da inversa

O artigo não específica o método adotado para obter a inversa, neste caso é utilizado a técnica Hensel lifiting

In [131]:
# def exponentiation(base, exp):
#     result = 1
#     current_power = base
#     current_exponent = exp

#     while current_exponent > 0:
#         if current_exponent % 2 == 1:
#             result *= current_power
#         current_power *= current_power
#         current_exponent //= 2
#     return result

# def S2_inverse(S2, a):
#     # aa = S2(a.lift())^(2**508 - 2)
#     aa = exponentiation(S2(a.lift()), 2**507 - 1)
#     aa = aa^2
#     return aa

Para melhorar a eficiência será utilizado outro algoritmo baseado no paper "O cálculo da inversa é baseado no paper "Parallel Itoh-Tsujii Multiplicative Inversion Algorithm for a Special Class of Trinomials". Baseado na função;
\begin{equation}
    \beta_{k+j}(a) = \beta_k(a)^{2^j}\beta_j(a)
\end{equation}


In [132]:
def exponentiation_hps(base):
    beta = {}
    beta[1] = base

    #elevar por 2^j
    def square_pow(x, j):
        for _ in range(j):
            x = x * x
        return x

    # A Shortest Addition Chain for n= 507, escrita como (k, j) com k+j = índice
    chain = [
        (2,1),(3,3),(6,6),(12,3),(15,15),(30,30),   
        (60,3),(63,63),(126,126),(252,252),(504,3),
    ]
    
    beta[2] = square_pow(beta[1],1)*beta[1]
    for k, j in chain:
        beta[k+j] = square_pow(beta[k],j)*beta[j]

    return beta[507]

def S2_inverse(S2, a):
    aa = exponentiation_hps(S2(a.lift()))
    aa = aa^2 # aa = S2(a.lift())^(2**508 - 2)
    
    return aa

In [133]:
def Sq_inverse(S2, Sq, a):
    # v0 = S2(a.lift())^(-1)
    v0 = S2_inverse(S2, a)
    v0 = Sq(v0.lift())
    for i in range(4):
        v0 = Sq(v0 * (2 - a * v0))
    return v0

## Geração de chaves (KeyGen)

O cálculo da inversa é necessário apenas na etapa de geração de chaves, como mostrado em "DPKE_Public_Key". 

O DPKE_Public_Key chama a função sample_fg para amostrar os polinômios $f$ e $g \in S$
\begin{align*}
    f \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    g \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

In [134]:
#section 1.10.1 (pg. 13)
def sample_fg(n, q):
    return ternary(n), ternary_fixed(n,q)

A partir das saidas podemos verificar, nesse caso como p é primo, fp é um corpo de galóis, ou seja sempre haverá inversa nesse anel, permitindo utilizar a função nativa do Sage

In [135]:
def DPKE_Key_Pair(S, S2, S3, Sq, Rq, n, q):
    for attempt in range(5):
        f, g = sample_fg(n, q)
        # fp = 1 / canonicalNTRU(S3, f.list(), n, q)
        fp = 1/S3(f)
        h, hq, v0, v1, G = DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g)

        print(f"[n,q]=[{n},{q}], attempt: {attempt + 1}")

        return f, g, fp, h, hq, v0, v1

SageMath representa fielmente o anel S2 das específicações do NTRU. Um dos possíveis erros abaixo, implementação antiga fornece grau n-2, sendo especificado n-1 para Rq

In [136]:
def DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g):
        try:
            G = 3 * g
            v0 = Sq(G * f)
            v1 = Sq_inverse(S2, Sq, v0)
            v1 = zz(v1)
            h = Rq(v1*G*G)
            hq = Rq(v1 * f * f)

            return h, hq, v0, v1, G

        except (ZeroDivisionError, ArithmeticError):
            pass

        raise ValueError("limite de tentativas em keygen()")


## Encrypt

In [137]:
def sample_rm(n, q):
    return ternary(n), ternary_fixed(n,q) 

In [138]:
def DPKE_Encrypt(Rq, h, r, m):
    h = zz(h)
    c = Rq(r*h + m)
    return c

## Decrypt

In [139]:
def decrypt(S3, Sq, Rq, c, f, fp, hq, q):
    c = zz(c)
    v1 = canonicalNTRU(Rq, (c*f).list(), n, q)
    fp = canonicalNTRU(S3, fp.list(), n, q)
    m0 = canonicalNTRU(S3, (v1 * fp), n, q)
    hq = zz(hq)
    r = canonicalNTRU(Sq, ((c-m0)*hq).list(), n, q)

    
    r_coeffs = r.list()
    m0_coeffs = m0.list()
    r_ok = all(c in {-1, 0, 1} for c in r_coeffs) and (r != 0)    
    weight = q // 8 - 2
    m0_ok = (all(c in {-1, 0, 1} for c in m0_coeffs) and 
             (m0 != 0) and
             sum(1 for c in m0_coeffs if c == 1) == weight // 2 and
             sum(1 for c in m0_coeffs if c == -1) == weight // 2)
    fail = 0 if (r_ok and m0_ok) else 1
    print("fail:", fail)
    return r, m0, fail

## Testes e profiling

In [140]:

class TestNTRU(unittest.TestCase):
    def setUp(self):
        self.qs = {}
        self.rings = {}
        self.keys = {}
        self.qs[509] = 2048
        self.ns = [509]
        self.p = 3
        
        for n in self.ns:
            q = self.qs[n]
            Z.<xZ> = ZZ[]                                   # ZZ[x]
            Z2.<xZ2> = Integers(2)[]                        # (ZZ/2ZZ)[x]
            Z3.<xZ3> = Integers(3)[]                        # (ZZ/3ZZ)[x]
            Zq.<xZq> = Integers(q)[]                        # (ZZ/2048ZZ)[x]
            Phi_n = lambda x : (x^n-1)//(x-1)
            Phi_1 = lambda x : (x - 1)
            R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
            S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
            R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
            Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
            S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
            S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
            Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
            R1.<xR1> = Zq.quotient(Phi_1(xZq))            

            self.rings[(n, q)] = {
                'S'    :  S,
                'Sq'   :  Sq,
                'Rq'   :  Rq,
                'S2'   :  S2,
                'S3'   :  S3,
                'R'    :  R,
                'Phi_n':  Phi_n,
                'R1'   :  R1
            }
            rings = self.rings[(n,q)]

            f, g, fp, h, hq, v0, v1 = DPKE_Key_Pair(rings['S'], rings['S2'], rings['S3'], rings['Sq'], rings['Rq'], n, q)
            
            self.keys[(n,q)] = {
                'f'  : f,
                'g'  : g,
                'fp' : fp,
                'h'  : h,
                'hq' : hq,
                'v0' : v0,
                'v1' : v1
            }

            r,m = sample_rm(n, q)
            
            c = DPKE_Encrypt(rings['Rq'], h, r, m)

            r, m0, fail = decrypt(S3, Sq, Rq, c, f, fp, hq, q)


class TestKeygen(TestNTRU):
    def test_parameters(self):
        for n in self.ns:
            q = self.qs[n]
            d = q//8 - 2
            self.assertTrue((q & (q - 1)) == 0)     # q is a power of 2: (2^n = q)
            self.assertGreaterEqual(q, 32)          # q is not small?
            self.assertLessEqual(d, 2*n/3)          # limit of d
            self.assertEqual(gcd(self.p, q), 1)     # p and q are coprime
            self.assertLessEqual(self.p, q)         # p equal or less q 

    def test_ternary(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                t = ternary(n)
                self.assertLessEqual(t.degree(), n-2)                      # t of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t.list()))    # t is ternary polinomial
                self.assertNotEqual(t, 0)                                       # t non-zero ternary polinomial

    def test_ternary_fixed(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                t_fixed = ternary_fixed(n, q)
                self.assertLessEqual(t_fixed.degree(), n-2)                       # t_fixed of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t_fixed.list()))        # t_fixed is ternary polynomial
                self.assertNotEqual(t_fixed, 0)                                           # t_fixed non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in t_fixed.list() if coef == 1), q//16-1)  # t_fixed have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in t_fixed.list() if coef == -1), q//16-1) # t_fixed have exactly (q//16-1) coefficients equal to -1           

    def test_sample_fg(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                f, g = sample_fg(n, q)

                self.assertLessEqual(f.degree(), n-2)                          # f of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in f.list()))        # f is ternary polinomial
                self.assertNotEqual(f, 0)                                           # f non-zero ternary polinomial
                self.assertLessEqual(g.degree(), n-2)                          # g of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in g.list()))        # g is ternary polynomial
                self.assertNotEqual(g, 0)                                           # g non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in g.list() if coef == 1), q//16-1)  # g have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in g.list() if coef == -1), q//16-1) # g have exactly (q//16-1) coefficients equal to -1
    
    def test_key_par(self):
        for n in self.ns:
            q = self.qs[n]
            keys = self.keys[(n, q)]
            rings = self.rings[(n, q)]
            self.assertEqual(keys['fp'] * rings['S3'](keys['f']), 1)

            
    def test_public_key(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]
            keys = self.keys[(n, q)]

            for _ in range(1):
                R1 = rings['R1']
                self.assertEqual(R1(keys['g']), 0) # output section 1.11.2 (notes)

                h= zz(keys['h'])
                f= keys['f']
                g= zz(keys['g'])

                v0 = zz(keys['v0'])
                prod = v0 * keys['v1']
                prod = canonicalNTRU(Sq, prod, n, q)
                self.assertEqual(prod, 1)

                hq = zz(keys['hq'])

                self.assertEqual(canonicalNTRU(Sq, (h * hq).list(), n, q), 1)
                self.assertEqual(Rq(h * f), Rq(3 * g))
                self.assertEqual(canonicalNTRU(Rq, (h * f).list(), n, q), (3 * keys['g']))

class TestDPKE_Encrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                r, m = sample_rm(n, q)

                self.assertLessEqual(r.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                
                self.assertLessEqual(m.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1
        
    def test_operations(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]
            keys = self.keys[(n,q)]

            for _ in range(1):
                r, m = sample_fg(n, q)
                c = DPKE_Encrypt(rings['Rq'], keys['h'], r, m)
                self.assertIs(c.parent(), rings['Rq']) # c is an element of Rq

class TestDPKE_Decrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                r, m = sample_rm(n, q)

                self.assertLessEqual(r.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                
                self.assertLessEqual(m.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1

            

In [141]:
unittest.main(argv=[''], exit=False)
t  = TestKeygen()
t.setUp() 
print("Profiling test_parameters:")
%prun -l 10 t.test_parameters()
t.setUp() 
print("Profiling test_ternary:")
%prun -l 10 t.test_ternary()
t.setUp() 
print("Profiling test_ternary_fixed:")
%prun -l 10 t.test_ternary_fixed() 
print("\nProfiling test_sample_fg:")
t.setUp()
%prun -l 20 t.test_sample_fg() 
print("\nProfiling test_inverse:")
t.setUp()
%prun -l 30 t.test_public_key()

t1 = TestDPKE_Encrypt()
t1.setUp() 
print("Profiling test_operations:")
%prun -l 10 t1.test_operations()

t2 = TestDPKE_Decrypt()
t2.setUp()
print("Profiling test_operations:")

.

[n,q]=[509,2048], attempt: 1
fail: 0
[n,q]=[509,2048], attempt: 1


..

fail: 0
[n,q]=[509,2048], attempt: 1
fail: 0


..

...

..

[n,q]=[509,2048], attempt: 1
fail: 0
[n,q]=[509,2048], attempt: 1
fail: 0


..
----------------------------------------------------------------------
Ran 9 tests in 1.775s

OK


[n,q]=[509,2048], attempt: 1
fail: 0
[n,q]=[509,2048], attempt: 1
fail: 0
[n,q]=[509,2048], attempt: 1
fail: 0
Profiling test_parameters:
 [n,q]=[509,2048], attempt: 1
fail: 0
Profiling test_ternary:
 [n,q]=[509,2048], attempt: 1
fail: 0
Profiling test_ternary_fixed:
 
Profiling test_sample_fg:
[n,q]=[509,2048], attempt: 1
fail: 0
 
Profiling test_inverse:
[n,q]=[509,2048], attempt: 1
fail: 0
 [n,q]=[509,2048], attempt: 1
fail: 0
Profiling test_operations:
 [n,q]=[509,2048], attempt: 1
fail: 0
Profiling test_operations:


         2786 function calls in 0.002 seconds

   Ordered by: internal time
   List reduced from 36 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 polynomial_quotient_ring_element.py:617(list)
      507    0.000    0.000    0.000    0.000 random.py:242(_randbelow_with_getrandbits)
        4    0.000    0.000    0.000    0.000 polynomial_ring.py:335(_element_constructor_)
        1    0.000    0.000    0.000    0.000 random.py:350(shuffle)
        1    0.000    0.000    0.000    0.000 random.py:454(choices)
        1    0.000    0.000    0.000    0.000 polynomial_quotient_ring_element.py:108(__init__)
        1    0.000    0.000    0.000    0.000 3141110661.py:4(ternary_fixed)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.001    0.001 2745119461.py:1(DPKE_Encrypt)
      699    0.000    0.000    0.000    0.